<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DSE-203:-Final-Project" data-toc-modified-id="DSE-203:-Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DSE 203: Final Project</a></span><ul class="toc-item"><li><span><a href="#Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag" data-toc-modified-id="Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Author: Adelle Driker, Bo Yan, Paul Cabasag</a></span></li><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Connect-to-Neo4j-Database" data-toc-modified-id="Connect-to-Neo4j-Database-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Connect to Neo4j Database</a></span></li><li><span><a href="#Query-from-neo4j-and-create-data-frame-for-Recipe-List" data-toc-modified-id="Query-from-neo4j-and-create-data-frame-for-Recipe-List-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Query from neo4j and create data frame for Recipe List</a></span></li><li><span><a href="#Edit-data-for-lowercase" data-toc-modified-id="Edit-data-for-lowercase-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Edit data for lowercase</a></span></li><li><span><a href="#Perform-extraction-of-nouns-from-summary" data-toc-modified-id="Perform-extraction-of-nouns-from-summary-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Perform extraction of nouns from summary</a></span></li><li><span><a href="#Load-in-foodon-data" data-toc-modified-id="Load-in-foodon-data-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Load in foodon data</a></span></li><li><span><a href="#Perform-exact-valuematching-between-nouns-and-ingredients" data-toc-modified-id="Perform-exact-valuematching-between-nouns-and-ingredients-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Perform exact valuematching between nouns and ingredients</a></span></li><li><span><a href="#Create-dictionary-to-store-all-matched-ingredients-items" data-toc-modified-id="Create-dictionary-to-store-all-matched-ingredients-items-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Create dictionary to store all matched ingredients items</a></span></li><li><span><a href="#Transfer-ingredients-matches-into-neo4j" data-toc-modified-id="Transfer-ingredients-matches-into-neo4j-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Transfer ingredients matches into neo4j</a></span></li><li><span><a href="#Extract-calorie,-fat,-and-protein-values" data-toc-modified-id="Extract-calorie,-fat,-and-protein-values-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Extract calorie, fat, and protein values</a></span></li><li><span><a href="#Transfer-calorie,-fat,-and-protein-into-neo4j" data-toc-modified-id="Transfer-calorie,-fat,-and-protein-into-neo4j-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Transfer calorie, fat, and protein into neo4j</a></span></li></ul></li></ul></div>

# DSE 203: Final Project
## Author: Adelle Driker, Bo Yan, Paul Cabasag

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship, NodeMatcher
import pandas as pd
import numpy as np 
from sklearn.metrics import precision_score, recall_score
from collections import Counter
import nltk
import re

## Connect to Neo4j Database

In [4]:
url="bolt://localhost:7687"
pwd="12345"
graph=Graph(url, auth=('neo4j', pwd))

## Query from neo4j and create data frame for Recipe List

In [5]:
query = """
        MATCH (n:Recipe) RETURN n.id, n.title, n.summary, n.readyInMinutes, n.pricePerServing, n.glutenFree, n.dairyFree, n.vegan
"""
df_recipelist=graph.query(query).to_data_frame()

#clean data from Recipe List 
df_recipelist=df_recipelist.dropna()

#check original data
df_recipelist

,n.id,n.title,n.summary,n.readyInMinutes,n.pricePerServing,n.glutenFree,n.dairyFree,n.vegan
0,636385.0,Buckwheat Banana Pancakes,The recipe Buckwheat Banana Pancakes can be ma...,30.0,134.63,True,False,False
1,637832.0,Chia Seed Pudding,"You can never have too many dessert recipes, s...",45.0,154.30,True,True,False
2,633078.0,Authentic Italian Biscotti,Authentic Italian Biscotti might be just the <...,45.0,9.73,False,True,False
3,632155.0,Almond Nut Butter,"Almond Nut Butter is a <b>gluten free, dairy f...",45.0,43.63,True,True,True
4,642178.0,Egg and Dairy Free Pancakes,If you have around <b>about 45 minutes</b> to ...,45.0,7.50,False,True,False
...,...,...,...,...,...,...,...,...
928,636393.0,Buckwheat Galette With An Egg,"You can never have too many side dish recipes,...",45.0,17.73,False,False,False
929,639597.0,Classic French Onion Soup,If you want to add more <b>Mediterranean</b> r...,45.0,293.89,False,False,False
930,633160.0,"Avocado Toast With Caramelized Sweet Onion, Gr...","Avocado Toast With Caramelized Sweet Onion, Gr...",45.0,247.74,False,True,True
931,652284.0,Molten Chocolate Liquor Cakes,Molten Chocolate Liquor Cakes might be just th...,45.0,94.52,True,False,False


## Edit data for lowercase

In [6]:
df_recipelist=df_recipelist.groupby('n.summary').head(912)
df_recipelist=df_recipelist.reset_index(drop=True)
df_recipelist['n.summary']=df_recipelist['n.summary'].str.lower()

## Perform extraction of nouns from summary

In [7]:
%%time
#perform extraction of nouns from summary
is_noun=lambda pos: pos[:2]=='NN'

nouns=[]
stop_words=['get', 'time', 'you', 'me', 'i', 'we', 'might', 'be', 'just', 'are', 'is', 'for', 'this', 'it','<b>', '/', '>', '<', 'minutes', 'calories', 'recipe', 'figure']

for i in range(df_recipelist.shape[0]):
    tokenized=nltk.word_tokenize(df_recipelist['n.summary'][i])
    nouns_text=[word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos) and word not in stop_words]
    nouns.append(nouns_text)
    
len(nouns)    

Wall time: 12.5 s


911

## Load in foodon data

In [8]:
foodon=pd.read_csv('foodon_trimmed.csv')
foodon

,FoodOn
0,seaweed
1,boar
2,pan frying
3,stem or spear vegetable
4,fruit used as vegetable
...,...
225,mammalian milk protein
226,canola or rapeseed oil
227,macaroni and cheese
228,mammalian milk or milk component


## Perform exact valuematching between nouns and ingredients

In [9]:
%%time
#perform exact valuematching between nouns and foodon
matches=[]

for i in range(len(nouns)):
    w_matches=[]
    for j in range(foodon.shape[0]):
        if foodon['FoodOn'][j] in nouns[i]:
            w_matches.append(foodon['FoodOn'][j])
    matches.append(w_matches)

Wall time: 1.2 s


In [10]:
#export results
df_recipelist.to_csv('foodonVM.csv')

#output results
df_recipelist['matches']=matches
df_recipelist

,n.id,n.title,n.summary,n.readyInMinutes,n.pricePerServing,n.glutenFree,n.dairyFree,n.vegan,matches
0,636385.0,Buckwheat Banana Pancakes,the recipe buckwheat banana pancakes can be ma...,30.0,134.63,True,False,False,[]
1,637832.0,Chia Seed Pudding,"you can never have too many dessert recipes, s...",45.0,154.30,True,True,False,[]
2,633078.0,Authentic Italian Biscotti,authentic italian biscotti might be just the <...,45.0,9.73,False,True,False,"[sugar, tiramisu]"
3,632155.0,Almond Nut Butter,"almond nut butter is a <b>gluten free, dairy f...",45.0,43.63,True,True,True,[]
4,642178.0,Egg and Dairy Free Pancakes,if you have around <b>about 45 minutes</b> to ...,45.0,7.50,False,True,False,[sugar]
...,...,...,...,...,...,...,...,...,...
906,636393.0,Buckwheat Galette With An Egg,"you can never have too many side dish recipes,...",45.0,17.73,False,False,False,[]
907,639597.0,Classic French Onion Soup,if you want to add more <b>mediterranean</b> r...,45.0,293.89,False,False,False,[]
908,633160.0,"Avocado Toast With Caramelized Sweet Onion, Gr...","avocado toast with caramelized sweet onion, gr...",45.0,247.74,False,True,True,[]
909,652284.0,Molten Chocolate Liquor Cakes,molten chocolate liquor cakes might be just th...,45.0,94.52,True,False,False,[]


## Create dictionary to store all matched ingredients items

In [11]:
%%time

match_dict = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict:
        match_dict[curr_id].append(df_recipelist['matches'][i])
    else:
        match_dict[curr_id] = df_recipelist['matches'][i]

match_dict

Wall time: 12 ms


{'Buckwheat Banana Pancakes': [],
 'Chia Seed Pudding': [],
 'Authentic Italian Biscotti': ['sugar', 'tiramisu'],
 'Almond Nut Butter': [],
 'Egg and Dairy Free Pancakes': ['sugar'],
 'Vegan Strawberry Shortcake served with Vegan Whipped Cream': ['sugar'],
 'Cake with lemon, rosewater and pistachios': ['sugar'],
 'Barbecue Chicken Cauliflower Couscous Bowls': ['chicken'],
 'Blueberry, Chocolate & Cocao Superfood Pancakes - Gluten-Free/Paleo/Vegan': [],
 'Amaretto Cheesecake': [],
 'Gluten Free Almond Blueberry Coffee Cake': [],
 'Almond Joy Protein Shake': ['beverage'],
 'Banana Milo Milkshake': ['beverage'],
 'Almond Horns': [],
 'Cranberry Crumb Bars': ['sugar'],
 '5 Minute Gluten Free Wonder Buns': [],
 'Chocolate Chip Coconut Muffins': [],
 'Snap Pea and Green Bean Salad with Arugula Pesto': [],
 'Salmon Caesar Salad': [],
 'Bleu Cheese Stuffed Wine Cherries With Honeycomb': ['chicken', 'beverage'],
 'Caramel Peanut Fudge Cake': [],
 'Raspberry Walnut Coffee Cake': ['sugar'],
 'Che

## Transfer ingredients matches into neo4j

In [14]:
#transfer foodon matches into neo4j
matcher = NodeMatcher(graph)

for key in match_dict:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("food_matches", "")
    graph.push(curr_node)
    
for key in match_dict:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.food_matches = $fm" % str(key), fm = str(match_dict[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.food_matches = $fm' % str(key), fm = str(match_dict[key])) 

## Extract calorie, fat, and protein values

In [ ]:
#functions
def find_all_places(text,word):
    word_places = []
    i=0
    while True:
        word_place = text.find(word,i)
        i+=len(word)+word_place
        if i>=len(text):
            break
        if word_place<0:
            break
        word_places.append(word_place)
    return word_places
def find_all_combination(text,start,end):
    start_places = find_all_places(text,start)
    end_places = find_all_places(text,end)
    combination_list = []
    for start_place in start_places:
        for end_place in end_places:
            print(start_place)
            print(end_place)
            if start_place>=end_place:
                continue
            combination_list.append(text[start_place:end_place])
    return combination_list

#calories per recipe
calories=[]
for i in df_recipelist['n.summary']:
    calories.append(re.findall(r'\d+', find_all_combination(i,'<b>','calories</b>')[0])[-1])
print(calories)

#fats per recipe
fats=[]
for i in df_recipelist['n.summary']:
    fats.append(re.findall(r'\d+', find_all_combination(i,'<b>','fat</b>')[1])[-1])
print(fats)

#protiens per recipe
proteins=[]
for i in df_recipelist['n.summary']:
    proteins.append(re.findall(r'\d+', find_all_combination(i,'<b>','protein</b>')[0])[-1])
print(proteins)   

In [ ]:
#adding calories and fats as columns to recipe dataframe
df_recipelist['calories']=calories
df_recipelist['fats']=fats
df_recipelist['proteins']=proteins

In [ ]:
%%time
#create dictionary for calories
match_dict_calories = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict_calories:
        match_dict_calories[curr_id].append(df_recipelist['calories'][i])
    else:
        match_dict_calories[curr_id] = df_recipelist['calories'][i]
        
match_dict_calories

In [ ]:
%%time
#create dictionary for fats
match_dict_fats = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict_fats:
        match_dict_fats[curr_id].append(df_recipelist['fats'][i])
    else:
        match_dict_fats[curr_id] = df_recipelist['fats'][i]
        
match_dict_fats

In [ ]:
%%time
#create dictionary for proteins
match_dict_proteins = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict_proteins:
        match_dict_proteins[curr_id].append(df_recipelist['proteins'][i])
    else:
        match_dict_proteins[curr_id] = df_recipelist['proteins'][i]
        
match_dict_proteins

## Transfer calorie, fat, and protein into neo4j

In [ ]:
#transfer calories into neo4j
matcher = NodeMatcher(graph)

for key in match_dict_calories:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("calories", "")
    curr_node.calories=match_dict_calories[key]
    graph.push(curr_node)
    
for key in match_dict_calories:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.calories = $cal" % str(key), cal = str(match_dict_calories[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.calories = $cal' % str(key), cal = str(match_dict_calories[key]))    

In [ ]:
#transfer fats into neo4j
matcher = NodeMatcher(graph)

for key in match_dict_fats:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("fats", "")
    curr_node.fats=match_dict_fats[key]
    graph.push(curr_node)
    
for key in match_dict_fats:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.fats = $fat" % str(key), fat = str(match_dict_fats[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.fats = $fat' % str(key), fat = str(match_dict_fats[key]))    

In [ ]:
#transfer proteins into neo4j
matcher = NodeMatcher(graph)

for key in match_dict_proteins:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("proteins", "")
    curr_node.proteins=match_dict_proteins[key]
    graph.push(curr_node)
    
for key in match_dict_proteins:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.proteins = $pro" % str(key), pro = str(match_dict_proteins[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.proteins = $pro' % str(key), pro = str(match_dict_proteins[key]))    